# Importación de librerias

In [ ]:
from scipy.spatial.distance import cityblock, euclidean
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.mixture import GaussianMixture
from scipy.spatial import distance

# Lectura de archivo

In [ ]:
path = "./data/cmu/keystroke.csv" 
df = pd.read_csv(path)
df.drop(['sessionIndex', 'rep'], axis=1, inplace=True)
subjects = df["subject"].unique()

# Metodo de distancias

In [ ]:
def euclideanDistance(a, b):
    return distance.euclidean(a.values, b.values)
    #return np.linalg.norm(a.values - b.values)

def manhattanDistance(a, b):
    return distance.cityblock(a.values, b.values)

def cosDistance(a,b):
    return distance.cosine(a.values, b.values)

#Distancias a calcular
distancesMethods = [1, 2, 3]

In [ ]:
df

# Separación de la data de entrenamiento y de prueba
Se toma de forma aleatoria el 80% de los registros de cada usuario para considerarlos como data de entrenamiento, mientras que el restante forma a pasar a la data de prueba

In [ ]:
train_users = []
test_users = []
for subject in subjects:
    genuine_user_data = df.loc[df["subject"] == subject, :]
            
    imposter_data = df.loc[df["subject"] != subject, :]

    train, test_genuine  = train_test_split(genuine_user_data, train_size = 0.8, random_state=43, shuffle=True)

    train_users.append(train)
    test_users.append(test_genuine)
    

train_users = pd.concat(train_users)
test_users = pd.concat(test_users)

# Cálculo de las distancias y evaluación
1. Se calcula la distancias entre cada vector del dataset del train con el vector del test
2. Se escoge el vector individidual del train con la menor distancia encontrada
3. Se chequea si el vector individidual corresponde con el usuario del test 
3. Se comienza a contar las veces que se acierta el usuario y las veces que no
5. Se imprime los resultados

In [ ]:
for distancesMethod in distancesMethods:
    countGood = 0
    countFalse = 0
    
    for i in range(test_users.shape[0]):
        value = []
        for j in range(train_users.shape[0]):
            if 1 == distancesMethod:
                value.append(euclideanDistance(train_users.iloc[j, 1:],  test_users.iloc[i, 1:]))
            elif 2 == distancesMethod:
                value.append(manhattanDistance(train_users.iloc[j, 1:],  test_users.iloc[i, 1:]))
            elif 3 == distancesMethod:
                value.append(cosDistance(train_users.iloc[j, 1:],  test_users.iloc[i, 1:]))
        idx = np.argmin(value)
        
        if train_users.iloc[idx]["subject"] == test_users.iloc[i]["subject"] :
            countGood += 1
            #print( "raw "+ str(i)+ " Total: "+str(countGood) + " / " + str(test_users.shape[0]) + "==> " +str(round(countGood*100/test_users.shape[0],2)) + "%")
        else:
            countFalse += 1
            #print( "raw "+ str(i)+ " false")
            
    print("----------------------------------------------------")
    if 1 == distancesMethod:
        print("Resumen distancia euclidiana:")
    elif 2 == distancesMethod:
        print("Resumen distancia Manhattan:")
    elif 3 == distancesMethod:
        print("Resumen distancia Coseno:")

# Resultados

In [ ]:
print("Total acertados "+str(countGood) + " / " + str(test_users.shape[0]) + "==> " +str(round(countGood*100/test_users.shape[0],2)) + "%")
print("Total equivocados "+str(countFalse) + " / " + str(test_users.shape[0]) +"==> " +str(round(countFalse*100/test_users.shape[0],2)) + "%")